In [6]:
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import  kanren.assoccomm as la
from kanren import run,var,fact

stemmer = WordNetLemmatizer()
def cleaner(document):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', document).strip()
    
#     # remove all single characters
#     document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
#     # Remove single characters from the start
#     document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    # Converting to Lowercase
    document = document.lower()
    re.sub('^his | his ',' donald trump ',document)
    stop = stopwords.words('english')
    stop.remove('who')
    stop.remove('when')
    document = document.split()
    document = [i for i in document if i not in stop]
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)

    return document


## name finder
def find_name(q):
    q = cleaner(q).split()
    for index,i in enumerate(q):
        if i == 'trump' or i =='madison' or i == 'knauss' or i =='frederick' or i =='kushner':
            return q[index-1] + ' ' +q[index]



In [ ]:
rule = '''
Fred Trump, the father of Donald Trump was (1905–1999) was born in New York, was a successful real estate developer in 
New York City.[39][40] Using his inheritance, Fred Trump and his mother Elizabeth founded Elizabeth Trump & Son.
Donald Trump later renamed it The Trump Organization and served as its chairman and president until assuming the 
office of U.S. President.[41] Fred Trumps is also the father of Fred Trump Jr, Maryanne Trump Barry, Elizabeth Trump Garu 
and Robert Trump.

Donald Trump has five children by three marriages, as well as nine grandchildren.[46][47] In 1977, 
Trump married Czech model Ivana Winklmayr (née Zelníčková), at Marble Collegiate Church in Manhattan, 
in a ceremony performed by the Reverend Norman Vincent Peale.[48][49][50][51] Donald Trump is the father of Eric Trump (1984),
and Ivanka Trump (b. 1981). Ivana became a naturalized United States citizen 
in 1988.[52] The couple divorced in 1992, following Trump's affair with actress Marla Maples.[53] 
In October 1993, Maples gave birth to Trump's daughter, who was named Tiffany in honor of high-end retailer
Tiffany & Company.[54] Maples and Trump were married two months later in December 1993.[55] They divorced in 
1999,[56] and Tiffany was raised by Marla in California.[57]   
'''.lower()

def sent_parser(sent):
    sent = sent.split()
    ind_of = sent.index('of')
    ind_is = sent.index('is')
    
    name1 = ' '.join(sent[:ind_is])
    name2 = ' '.join(sent[ind_of+1:])
    ind = sent.index('of') - 1
    relation = sent[ind]
    return name1, name2, relation


relationship = {}
for i in rule.split('.'):
    if len(i) <= 1 :
        continue
    name1,name2,relation =sent_parser(i)
    inner_relation = {name1:name2}
    if not relation in relationship:
        relationship[relation] = []
        
    relationship[relation].append(inner_relation)
    
    
## this code generates above rule from that dictionairy :D ## never mind this code below
# for i in relationship:
#     for j in relationship[i]:
#         name1 = list(j)[0]
#         name2 = j[name1]
#         relation = i
#         print('{} is the {} of {}.'.format(name1,relation,name2))

In [8]:
## Defining functions to parse through rules here
from kanren import Relation, facts, run, conde, var, eq
father = Relation()
mother = Relation()
husband = Relation()

# Check if 'x' is the parent of 'y'
def parent(x, y):
    return conde([father(x, y)], [mother(x, y)])

# Check if 'x' is the grandfather of 'y'
def grandparent(x, y):
    temp = var()
    return conde((parent(x, temp), parent(temp, y)))

# Check for sibling relationship between 'a' and 'b'
## temp is parent of x, and temp is parent of y...
## i.e, if donald is parent of eric, and donald is parent of barron, 
def sibling(x, y):
    temp = var()
    return conde((parent(temp, x), parent(temp, y)))

# Check if x is y's uncle
## temp is father of x, and, temp is grandfather of y
## fred is father of robert trump(x), and fred is grandfather of tiffany trump(y)
def uncle(x, y):
    temp = var()
    return conde((parent(temp, x), grandparent(temp, y)))

for item in relationship['father']:
    facts(father, (list(item.keys())[0], list(item.values())[0]))

for item in relationship['mother']:
    facts(mother, (list(item.keys())[0], list(item.values())[0]))

for item in relationship['husband']:
    facts(husband, (list(item.keys())[0], list(item.values())[0]))

x = var()

In [ ]:
 while True:
    q = raw_input("  How can I help you ? ")
    if q =='stop':
        print('Bye')
        break
    name  = find_name(q)
    q = cleaner(q).split()
    
    if  name == None:
        print('Cannot find names in the quesiton realted to trump. please be more specific.')
        continue
    else :
        print('Name detected, onto the query')
    
    print('the question is parsed as : ',q)
    ## finding parent
    if 'parent' in q or 'father' in q or 'mother' in q:
        output = run(0, x, parent(x,name ))
        if len(output) == 0:
            print('Sorry. I cant find the answer.')
            continue
        else :
            print("\nList of " + name + "'s parent:")
            for item in output:
                print(item)
            continue
    
    ## finding grandparent
    if 'grandparent' in q or 'grandfather' in q or 'grandmother' in q:
        output = run(0, x, grandparent(x,name ))
        if len(output) == 0:
            print('Sorry. I cant find the answer')
            continue
        else :
            print("\nList of " + name + "'s grandparents:")
            for item in output:
                print(item)  
            continue
    
    ## finding grandchildren
    if 'grandchild' in q or 'grandchildren' in q or 'grandkids' in q:
        output = run(0, x, grandparent(name,x ))
        if len(output) == 0:
            print('Sorry. I cant find the answer')
            continue
        else :
            print("\nList of " + name + "'s grandchildren:")
            for item in output:
                print(item)    
            continue    
                    
    
    ## rule for finding uncle and aunt
    if 'uncle' in q or 'aunt' in q:
        try : 
            name_father = run(0, x, father(x, name))[0]
            output = run(0, x, uncle(x, name))
#             output = [x for x in output if x != name_father]
            print("\nList of " + name + "'s uncles and aunts:")
            for item in output:
                print(item)
            continue
        except:
            print('Sorry, I cant understand you')
            continue
    ## rule for finding nephew
    if 'nephew' in q or 'nephewness' in q :
        try :
            name_sons = run(0, x, father(name, x))[0]
            output = run(0, x, uncle(name, x))
#             output = [x for x in output if x not in name_sons]
            print("\nList of " + name + "'s nephew's:")
            for item in output:
                print(item)
            continue
        except :
            print('Sorry I cant understand you')
            continue
    ## rule for finding children .
    if 'child' in q or 'children' in q:
        output = run(0, x, parent(name,x ))
        if len(output) == 0 :
            print('Sorry I cant understand you')
            continue

        else :
            print("\nList of " + name + "'s children:")
            for item in output:
                print(item)
            continue
                ##  the values coming from rules
    ## finding siblings rule
    # find someone's siblings
    if 'sibling' in q or 'brother' in q or 'sister' in q:
         ##"Donald Trump" or "Ivanka Trump" or "Eric Trump"
        output = run(0, x, sibling(name,x ))
        if len(output) == 0:
            print('Sorry I cant understand you')
            continue
        else :
#             output = [x for x in output if x != name]
            print("\nList of " + name + "'s Siblings:")
            for item in output:
                print(item)
            continue

  How can I help you ? 
Cannot find names in the quesiton realted to trump. please be more specific.
  How can I help you ? who is donald trump father
Name detected, onto the query
('the question is parsed as : ', ['who', 'donald', 'trump', 'father'])

List of donald trump's parent:
fred trump
mary ann trump
  How can I help you ? who is husband of melania 
Cannot find names in the quesiton realted to trump. please be more specific.
  How can I help you ? who are children of melania knauss
Name detected, onto the query
('the question is parsed as : ', [u'who', u'child', u'melania', u'knauss'])

List of melania knauss's children:
barron trump
  How can I help you ? Who is the uncle of eric trump
Name detected, onto the query
('the question is parsed as : ', ['who', 'uncle', 'eric', 'trump'])

List of eric trump's uncles and aunts:
elizabeth trump garu
donald trump
maryanne trump barry
robert trump
fred trump jr
  How can I help you ? Who is eric trump uncle
Name detected, onto the query